# Feature Extraction
This notebook will focus on extracting features from the preprocessed text data using TF-IDF, Word2Vec, and Sentence Transformers. The extracted featurees will be used in the subsequent notebook for model training and evaluation.  

## Set Up Dependencies

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer

Let's load the preprocessed dataframe from the previous notebook.

In [3]:
df_balanced = pd.read_csv('data/df_balanced.csv')
df_balanced.head(10)

,class,tweet,cleaned_text
0,0,I LOVE my 10 &amp; 5 but most days they remind...,"['love', 'my', 'amp', 'but', 'most', 'days', '..."
1,1,She be thinking she throwing that pussy back s...,"['she', 'be', 'thinking', 'she', 'throwing', '..."
2,1,RT @lamessican: I love when bitches throw shad...,"['love', 'when', 'bitches', 'throw', 'shade', ..."
3,1,"If you ain't a hoe, get up out my trap house @...","['if', 'you', 'ain', 'hoe', 'get', 'up', 'out'..."
4,0,Just hit 40 in flappy bird.&#128527;,"['just', 'hit', 'in', 'flappy', 'bird']"
5,0,WEBBERS! RT @DMILZ4: Sz13 vnds restorers dream...,"['sz', 'vnds', 'restorers', 'dream', 'cracked'..."
6,0,RT @jemelehill: Just saw Dawn of The Planet of...,"['just', 'saw', 'dawn', 'of', 'the', 'planet',..."
7,1,RT @femaIes: I swear &#128514; we in this fore...,"['swear', 'we', 'in', 'this', 'forever', 'now'..."
8,0,"Moral to the story: If you like the Yanks, I'l...","['if', 'you', 'like', 'the', 'yanks', 'll', 'p..."
9,1,"Dayum u gyp, u gone have mfs thinking im wipin...","['dayum', 'gyp', 'gone', 'have', 'mfs', 'think..."


## Prepare Data for Feature Extraction

We split the data into a 80/20 train-test split.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_balanced['cleaned_text'], df_balanced['class'], test_size=0.2, random_state=42)

## TF-IDF